In [1]:
import torch
import tqdm
import pandas as pd
from torcheval.metrics import BinaryConfusionMatrix
import numpy as np

In [4]:
def CustomDataframe(samples,features,func,normal=False):
    x =torch.rand([samples,features+1])
    y=torch.where( func(x[:,:-1]) ,1,0)
    if normal:            
        x=x[~ y.bool()]
        samples=x.shape[0]
        x[:,-1]=0
    else:
        x[:,-1]=y
    print(x[:,-1].mean())
    print(samples,y.shape)
    df=pd.DataFrame(x)
    l=[f"col_{i}" for i in range(features)]
    l.append("attack")
    df.columns=l
    return df

In [2]:
def NonLinear(x:torch.Tensor):
    return (x[:, 0]**2+x[:, -1]**2-x[:, 0] * x[:, -1])>.7

In [5]:
samples=10000
features=5

noraml_df=CustomDataframe(samples,features,NonLinear,True)
abnoraml_df=CustomDataframe(samples,features,NonLinear,False)


tensor(0.)
8462 torch.Size([10000])
tensor(0.1479)
10000 torch.Size([10000])


## DECESION TREE
works great here

In [6]:
from sklearn.tree import DecisionTreeClassifier

dt=DecisionTreeClassifier(random_state=32)
df=abnoraml_df.copy(deep=True)
y=df['attack']
df.drop(columns=['attack'])
dt.fit(X=df,y=y)

DecisionTreeClassifier(random_state=32)

In [7]:
dfn=noraml_df.copy(deep=True)
yn=dfn['attack']
dfn.drop(columns=['attack'])

,col_0,col_1,col_2,col_3,col_4
0,0.733178,0.695553,0.633435,0.500656,0.737202
1,0.387885,0.263694,0.821976,0.771281,0.686281
2,0.106234,0.418982,0.834129,0.099004,0.031417
3,0.427075,0.995037,0.046403,0.964152,0.032465
4,0.346969,0.835123,0.084330,0.950386,0.669154
...,...,...,...,...,...
8457,0.874422,0.742087,0.072422,0.534825,0.546723
8458,0.815274,0.294159,0.923171,0.234382,0.408409
8459,0.052138,0.637868,0.565389,0.831830,0.760738
8460,0.382591,0.883459,0.125626,0.014019,0.469120


In [8]:
from sklearn.metrics import confusion_matrix
yp=dt.predict(X=df)
confusion_matrix(y,yp)

array([[8521,    0],
       [   0, 1479]], dtype=int64)

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, samples,features,func,normal=False):
        self.samples=samples
        self.features=features
        self.x =torch.rand([samples,features])
        self.y=torch.where( func(self.x) ,1,0)
        if normal:            
            self.x=self.x[~ self.y.bool()]
            self.samples=self.x.shape[0]
            self.y=torch.zeros(self.samples)
        self.y=self.y.float()
        print(self.y.mean(-1))
        print(samples,self.y.shape)

    def __len__(self):
        return self.samples
    def __getitem__(self, idx):
        return self.x[idx].unsqueeze(-1).cuda(),self.y[idx].unsqueeze(-1).cuda(),self.y[idx].unsqueeze(-1).cuda()

In [3]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='./tmp/Cora', name='Cora')

Processing...
Done!


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        self.adj=dense_to_sparse(torch.ones(2708,2708))[0].cuda()
        self.w=torch.nn.Parameter(torch.rand(2708*2708,requires_grad=True))
        self.w.requires_grad=True

    def forward(self, data):
        x, _ = data.x, data.edge_index
        if x.dim()==3:
            batch=x.shape[0]
            edge_index=self.adj.repeat(batch)
            edge_w=self.w.repeat(batch)
        else:
            edge_index=self.adj
            edge_w=self.w      
        x = self.conv1(x, edge_index=edge_index,edge_weight=edge_w)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index=edge_index,edge_weight=edge_w)

        return F.log_softmax(x, dim=1)

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
model.w.sum()


tensor(3666371., device='cuda:0', grad_fn=<SumBackward0>)

In [27]:

data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(50):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [28]:
model.w.sum()

tensor(1151770.7500, device='cuda:0', grad_fn=<SumBackward0>)

In [20]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.1300


In [11]:
torch.arange(3).float().requires_grad_(True).repeat(3)

tensor([0., 1., 2., 0., 1., 2., 0., 1., 2.], grad_fn=<RepeatBackward0>)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [81]:
class HiddenLayer(torch.nn.Module):
    def __init__(self,features):
        super(HiddenLayer, self).__init__()    
        self.first = torch.nn.Linear(features, features)  # Input size is 2 (1st and last features), 64 hidden units
        self.relu = torch.nn.ReLU()  

    def forward(self, x):
        x = self.first(x)
        x = self.relu(x)
        return x
class SimpleModel(torch.nn.Module):
    def __init__(self,features,embeding,hidden,hidden_layers):
        super(SimpleModel, self).__init__()
        # Define a neural network with a few layers to learn nonlinear relationships
        self.enc=torch.nn.Sequential(
            torch.nn.Linear(features, embeding),
            torch.nn.ReLU(),
            torch.nn.Linear(embeding, hidden),
            torch.nn.ReLU()
        )
        self.layers = torch.nn.ModuleList(
            [
                HiddenLayer(hidden)
                for i in range(hidden_layers)
            ]
        )
        self.dec=torch.nn.Sequential(
            torch.nn.Linear(hidden, embeding),
            torch.nn.ReLU()
        )
        self.last = torch.nn.Linear(embeding, 1)  # Output layer, 1 unit (binary classification)
        self.sigmoid = torch.nn.Sigmoid()  # Sigmoid for binary classification output

    def forward(self, x):
        x = self.enc(x)
        for layer in self.layers:
            x = layer(x)
        x = self.dec(x)
        x = self.last(x)
        x = self.sigmoid(x)
        return x

In [82]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse
from src.models.BaseModel import BaseModel

class GCN(torch.nn.Module):
    def __init__(self,features,embeding,hidden,hidden_layers):
        super().__init__()
        self.node_num=features
        self.conv1 = GCNConv(1, embeding)
        self.conv2 = GCNConv(embeding, hidden)
        self.layers = torch.nn.ModuleList(
            [
                GCNConv(hidden, hidden)
                for i in range(hidden_layers)
            ]
        )
        self.conv3 = GCNConv(hidden, 1)
        self.last = torch.nn.Linear(features, 1)  # Output layer, 1 unit (binary classification)
        self.sigmoid = torch.nn.Sigmoid()  # Sigmoid for binary classification output
        self.edge_index=torch.tensor([[0,features-1],[features-1,0]]).cuda()

    def forward(self, data:torch.Tensor):
        x = self.conv1(data, self.edge_index)
        x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        x = self.conv2(x, self.edge_index)
        for layer in self.layers:
            x = layer(x, self.edge_index)
            x = F.relu(x)
            # x = F.dropout(x, training=self.training)
        x = self.conv3(x, self.edge_index)
        x = F.relu(x)
        x = self.last(x)
        x = self.sigmoid(x)
        return x
        
class GCN_AT(BaseModel):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)

        self.node_embeddings=torch.nn.Parameter(
            torch.rand(
                self.node_num,
                self.param.embedding_dimension                )
            )
        
        self.conv_first = GCNConv(1, self.param.out_layer_inter_dim)
        self.layers = torch.nn.ModuleList(
            [
                GCNConv(self.param.out_layer_inter_dim, self.param.out_layer_inter_dim)
                for i in range(self.param.out_layer_num)
            ]
        )
        self.conv_last = GCNConv(self.param.out_layer_inter_dim,1)
        self.init_parameters()
        
    def getAdj(self):
        """
        Returns:
            adj_matrix (Tensor): Sparse adjacency matrix (N, N)
        """
        # Normalize embeddings for stable cosine similarity
        normalized_embeddings = F.normalize(self.node_embeddings, p=2, dim=1)
        
        # Compute cosine similarity matrix
        sim_matrix = torch.mm(normalized_embeddings, normalized_embeddings.t())
        
        # Ensure self-similarity is not selected
        sim_matrix.fill_diagonal_(-1)  # Set diagonal to -1 to exclude self-connections
        
        # # Get top-k neighbors for each node
        topk_values, topk_indices = torch.topk(sim_matrix, k=self.param.topk, dim=1)
        
        # # Create binary adjacency mask
        adj_mask = torch.zeros_like(sim_matrix)
        adj_mask.scatter_(1, topk_indices, 1.0)
        
        # For undirected graphs, ensure symmetric connections
        # if not self.directed:
        # adj_mask = torch.max(sim_matrix, sim_matrix.t())
        # adj_mask.fill_diagonal_(1)  # Add self-loops
        
        return adj_mask 
    
    def init_parameters(self):
        torch.nn.init.xavier_uniform_(self.node_embeddings)   
    
    def pre_forward(self, data:torch.Tensor):
        adj,_=dense_to_sparse(self.getAdj())
        x = self.conv_first(data, adj)
        x = F.relu(x)
        for layer in self.layers:
            x = layer(x, adj)
            x = F.relu(x)
            # x = F.dropout(x, training=self.training)
        x = self.conv_last(x, adj)
        x = F.relu(x)
        # x = self.last(x.squeeze(-1))
        # x = self.sigmoid(x)
        return x

In [83]:
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    total = 0
    correct = 0
    model.cuda()
    metric=BinaryConfusionMatrix()
    with torch.no_grad():  # Disable gradient computation for testing
        t=tqdm.tqdm(test_loader,leave=False)
        for features, labels in t:
            # Forward pass
            outputs = model(features.detach().cuda())
            
            # Assuming binary classification: outputs are probabilities
            # predicted = torch.where((outputs > 0.5),1,0).float()  # Convert probabilities to binary predictions
            metric.update(outputs.squeeze(),labels.squeeze())
            # Calculate accuracy
            # total += labels.size(0)
            # correct += (predicted.cpu() == labels).sum().item()
    r=metric.compute().tolist()
    # accuracy = 100 * correct / total
    return r[0][0]/(r[0][0]+r[0][1]),r[1][1]/(r[1][0]+r[1][1])

In [84]:
# Example training loop
def train_model(model:torch.nn.Module, 
                train_data_loader:torch.utils.data.DataLoader, 
                test_data_loader:torch.utils.data.DataLoader,
                epochs:int):
    criterion = torch.nn.MSELoss() # Binary Cross-Entropy Loss for classification
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    model=model.train().cuda()
    t=tqdm.tqdm(torch.arange(epochs))
    t.set_postfix_str("{Loss:.4f}")
    train_p_recall=[]
    train_n_recall=[]
    test_p_recall=[]
    test_n_recall=[]   
    _loss=[]
    for epoch in t:
        for features, labels in train_data_loader:
            # Extract the 1st and last features only
            # x = features[:, [0, -1]]  # Select 1st and last feature
            # labels = labels.view(-1, 1)  # Reshape labels for BCELoss
            
            # Forward pass
            outputs = model(features.cuda())
            loss = criterion(outputs, labels.cuda().float())
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        t.set_postfix({"Loss": loss.item()})
        _loss.append(loss.item())
        rtrain=test_model(model,train_data_loader)
        rtest=test_model(model,test_data_loader)
        train_p_recall.append(rtrain[0])
        train_n_recall.append(rtrain[1])
        test_p_recall.append(rtest[0])
        test_n_recall.append(rtest[1])
        
    return pd.DataFrame({"train_p_recall":train_p_recall,
                         "train_n_recall":train_n_recall,
                         "test_p_recall":train_p_recall,
                         "test_n_recall":test_n_recall,
                         "loss":_loss})
        

# Note: You need to provide a data loader (`data_loader`) that yields batches of your dataset.


In [85]:
from train_loop import train
from util.params import Params
from util.consts import Tasks
from util.env import set_param
from datasets.TimeDataset import TimeDataset
from datasets.CurrentDataset import CurrentDataset
sensors=noraml_df.columns.drop("attack")

param=Params()
param.epoch=5
param.task=Tasks.current_label
param.batch=16
param.topk=2
set_param(param)


normal_dataset=CurrentDataset(noraml_df,sensors,[])
abnormal_dataset=CurrentDataset(abnoraml_df,sensors,[],mode="test")



In [86]:
abnormal_dataset.__getitem__(0)

(tensor([[0.0693],
         [0.3560],
         [0.4138],
         [0.1085],
         [0.7405]], device='cuda:0'),
 tensor([], device='cuda:0', size=(0, 1)),
 tensor([0.], device='cuda:0'))

In [87]:
import torch_geometric
import torch_geometric.data
import torch_geometric.nn

class GAT_PG(torch.nn.Module):
    """docstring for GAT_PG."""
    def __init__(self, **kwarg):
        super(GAT_PG, self).__init__(**kwarg)
        self.gat=torch_geometric.nn.GCNConv(1,1)
        self.lin=torch.nn.Linear(features,1)
        self.adj=torch.nn.Parameter(dense_to_sparse(torch.ones(features,features))[0],requires_grad=False)
        self.w=torch.nn.Parameter(torch.rand(features*features),requires_grad=True)
        

    def forward(self,x:torch.Tensor):
        x=x.detach().clone()
        if x.dim()==3: #batched
            batch=x.shape[0]
            xl=[torch_geometric.data.Data(x=x_,edge_index=self.adj,edge_attr=self.w) for x_ in x]
            batched=torch_geometric.data.Batch.from_data_list(xl)
            x=self.gat(batched.x,edge_index=batched.edge_index,edge_weight=batched.edge_weight)
            x=x.view(batch,-1)
        else:
            x=self.gat(x,edge_index=self.adj,edge_weight=self.w)
            x=x.squeeze(-1)
        x=self.lin(x)
        x=F.sigmoid(x)
        return x

In [88]:
class GAT_PGDS(torch.nn.Module):
    def __init__(self, features):
        super(GAT_PGDS, self).__init__()
        self.gat = torch_geometric.nn.GCNConv(1, 1)
        self.lin = torch.nn.Linear(features, 1)
        adj_matrix = torch.ones(features, features).cuda()
        self.adj = dense_to_sparse(adj_matrix)[0]  # edge_index shape [2, num_edges]
        self.num_edges = self.adj.size(1)
        self.w = torch.nn.Parameter(torch.rand(self.num_edges), requires_grad=True)
        
    def forward(self, x: torch.Tensor):
        if x.dim() == 3:  # Batched input [batch_size, num_nodes, 1]
            batch_size, num_nodes, _ = x.shape
            x = x.view(-1, 1)  # Flatten to [batch_size * num_nodes, 1]
            
            # Adjust edge_index for each graph in the batch
            offsets = torch.arange(batch_size, device=x.device) * num_nodes
            edge_index = (self.adj.unsqueeze(1) + offsets.view(-1, 1, 1)).view(2, -1)
            
            # Repeat self.w for each graph in the batch
            edge_weight = self.w.repeat(batch_size)
            
            x = self.gat(x, edge_index, edge_weight)
            x = x.view(batch_size, -1)
        else:
            x = self.gat(x, self.adj, self.w)
            x = x.squeeze(-1)
        
        x = self.lin(x)
        x = torch.sigmoid(x)
        return x

In [89]:
normal_loader=torch.utils.data.DataLoader(normal_dataset,param.batch,shuffle=True)
abnormal_loader=torch.utils.data.DataLoader(abnormal_dataset,param.batch)
# Instantiate the model, loss function, and optimizer
# model = SimpleModel(features,64,32,0)
model=GAT_PG()#GCN_AT(node_num=features)
model=model.cuda()
model=model.train()


In [97]:
model(normal_dataset.__getitem__(0)[0].cuda())

tensor([0.5235], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [101]:
l=model(normal_dataset.__getitem__(0)[0].cuda())-.53


In [ ]:
l.

C:\Users\moji\AppData\Local\Temp\ipykernel_16344\3815194156.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:494.)
  l.grad


In [99]:
print(model.w.grad)

None


In [91]:
with torch.no_grad():
    print(model.w.sum())
    
    a,w=dense_to_sparse(model.adj)
    print(a[0].shape)
    print(model.w)

tensor(11.2446, device='cuda:0')
torch.Size([40])
Parameter containing:
tensor([0.4915, 0.3565, 0.0060, 0.1899, 0.5869, 0.6925, 0.7799, 0.5875, 0.0289,
        0.9239, 0.7400, 0.7563, 0.4715, 0.0762, 0.4811, 0.0664, 0.2827, 0.3466,
        0.9487, 0.4724, 0.1966, 0.5589, 0.5069, 0.3215, 0.3752],
       device='cuda:0', requires_grad=True)


In [92]:
for name,p in model.named_parameters():
    print(name,p.shape , p.requires_grad)

adj torch.Size([2, 25]) False
w torch.Size([25]) True
gat.bias torch.Size([1]) True
gat.lin.weight torch.Size([1, 1]) True
lin.weight torch.Size([1, 5]) True
lin.bias torch.Size([1]) True


In [72]:
import tqdm

optimizer = torch.optim.Adam(
    model.parameters(), lr=param.learning_rate, weight_decay=param.decay
)

acu_loss = 0
min_loss = 1e8
min_train_loss = 1e8

epoch = param.epoch
early_stop_win = 15
model.to(param.device)

stop_improve_count = 0

acu_loss = 0
loss_func = torch.nn.L1Loss()  # param.loss_function()

for epoch in range(param.epoch):
    t=tqdm.tqdm(normal_loader,postfix="{loss}",)
    acu_loss=0
    for b,(data) in enumerate(t):
        y=model(data[0])
        loss=loss_func(y,data[2])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acu_loss += loss.item()
        t.set_postfix({"loss":loss.item()})
    t.set_postfix({"loss":loss.item()})
    t.close()


 12%|█▏        | 63/534 [00:00<00:05, 80.59it/s, loss=8.37e-5] 


KeyboardInterrupt: 

In [48]:
with torch.no_grad():
    print(model.w.sum())
    print(model.w)

tensor(13.1259, device='cuda:0')
Parameter containing:
tensor([0.7120, 0.4131, 0.6247, 0.2921, 0.5111, 0.1222, 0.6396, 0.4460, 0.4027,
        0.7959, 0.2787, 0.8526, 0.6257, 0.7505, 0.6336, 0.8621, 0.7057, 0.4243,
        0.2755, 0.4535, 0.0611, 0.6223, 0.3871, 0.7866, 0.4474],
       device='cuda:0', requires_grad=True)


In [ ]:
train(model=model,train_dataloader=normal_loader,val_dataloader=abnormal_loader)

epoc 1 / 5:   0%|          | 0/536 [00:00<?, ?it/s, {loss} {val_loss}]

In [ ]:

result=train_model(model=model,
                   train_data_loader=normal_loader,
                   test_data_loader=abnormal_loader,
                   epochs=50)
result.iloc[-1]

In [ ]:
yp.squeeze().mean()

In [ ]:

model.eval()
yp=model(noram_dataset.x.cuda())
c=BinaryConfusionMatrix()
c.update(noram_dataset.y.detach(),yp.detach().squeeze())
c.compute()

In [ ]:
result.loc[150:,"loss"].mean()

In [ ]:
result.plot(y=["loss"])

In [ ]:
result.iloc[:].plot()

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

dataset = Planetoid('/tmp/Planetoid', 'Cora', transform=T.NormalizeFeatures())
data = dataset[0]


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.edge_weight = torch.nn.Parameter(torch.ones(data.num_edges))
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16,1)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index, self.edge_weight.sigmoid()).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, self.edge_weight.sigmoid())
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    F.nll_loss(out[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    print(model.edge_weight[:5])
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, best_val_acc, test_acc))